```
Group: 3
Topic: Fire Detection Using Multi Color Space and Background Modeling
Group Members:
      IIT2018047
      IIT2018050
      IIT2018054
```




Imports

In [ ]:
import numpy as np
import cv2

Global Var

In [ ]:
# global const for rectangle
min_x, min_y = 100000, 100000
max_x = max_y = 0

# global const for intensity
FP = 0
lastFP = 0

# font
font = cv2.FONT_HERSHEY_SIMPLEX

# org
org = (20, 210)

# fontScale
fontScale = 0.5

# Blue color in BGR
color = (255, 0, 0)

# Line thickness of 2 px
thickness = 2

Image enhancing

In [ ]:
def enhancing(image):
    image[:, :, 0] = 255 * pow(image[:, :, 0] / 255, 0.9)
    image[:, :, 1] = 255 * pow(image[:, :, 1] / 255, 0.7)
    image[:, :, 2] = 255 * pow(image[:, :, 2] / 255, 1.5)

    return image

Fire Pixel Detection

In [ ]:
def alg(image, frame_img, fg_bg):
    global max_x, max_y, min_x, min_y, FP
    redTh = 135
    saturationTh = 55

    lab = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)

    B = image[:, :, 0]
    G = image[:, :, 1]
    R = image[:, :, 2]

    L = lab[:, :, 0]
    a = lab[:, :, 1]
    b = lab[:, :, 2]

    S = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)[:, :, 1]

    R_mean = np.mean(R)

    # Find the mean of the L* A* B* values
    L_mean, a_mean, b_mean, x = cv2.mean(lab)

    # Check all the Rules of the Algorithm for RGB
    fire_area_rgb = np.array(np.where(R > R_mean,
                                      np.where(R > G,
                                               np.where(G > B,
                                                        np.where(S > (255 - R) * saturationTh / redTh, 255,
                                                                 0), 0), 0), 0))

    # Check all the Rules of the Algorithm for Lab
    fire_area_lab = np.array(np.where(L >= L_mean,
                                      np.where(a >= a_mean,
                                               np.where(b >= b_mean,
                                                        np.where(b >= a, 255,
                                                                 0), 0), 0), 0))

    # Removing Background
    fgmask = fg_bg.apply(frame_img)

    # counting fire pixels
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if (fire_area_rgb[i][j] != 0 or fire_area_lab[i][j] != 0) and fgmask[i][j] != 0:
                max_x = max(max_x, j)
                max_y = max(max_y, i)
                min_x = min(min_x, j)
                min_y = min(min_y, i)
                FP += 1


to Find intensity

In [ ]:
def rateOfGrowth():
    global FP, lastFP
    rate = (FP - lastFP) / 1
    lastFP = FP
    FP = 0
    return rate

main function

In [ ]:
if __name__ == '__main__':
    cap = cv2.VideoCapture("barbeq.avi")
    fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False)

    if cap.isOpened():
        width = int(cap.get(3))
        height = int(cap.get(4))

        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter('output.avi', fourcc, 20.0, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if ret:

            # for each time max and min
            min_x, min_y = 100000, 100000
            max_x = max_y = 0

            enhanced_image = enhancing(frame.copy())

            output = alg(enhanced_image, frame, fgbg)
            cv2.rectangle(frame, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)

            # Using cv2.putText() method
            frame = cv2.putText(frame, str(rateOfGrowth()), org, font, fontScale, color, thickness, cv2.LINE_AA)

            # write the frame
            out.write(frame)

        else:
            break

    # Release everything if job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()